# ETL: Raw → Silver Layer

## Contexto do Projeto

Este notebook realiza o processo completo de ETL (Extract, Transform, Load) dos dados brutos de filmes (TMDB) para a camada Silver do Data Lake.

### Persona: Diretor de Estratégia de Investimentos

**Objetivo:** Identificar o próximo filme de sucesso com o menor orçamento possível, maximizando o ROI (Return on Investment).

**Questões de Negócio:**
- Quais gêneros apresentam melhor relação custo-benefício?
- Qual a faixa de orçamento ideal para maximizar o lucro?
- Quais produtoras têm histórico de filmes rentáveis?
- Em qual época do ano lançar para maximizar receita?

---

## Processo ETL

1. **Extract**: Carrega dados brutos do arquivo CSV
2. **Transform**: Aplica limpeza, filtragem e criação de métricas derivadas
3. **Load**: Carrega dados tratados no PostgreSQL (schema `silver`) e gera backup CSV

## 1. Configuração e Imports

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_batch
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 80)

In [2]:
# =============================================================================
# CONFIGURAÇÕES DO PROJETO
# =============================================================================

# Caminhos dos arquivos
INPUT_FILE = '../Data Layer/raw/dados_brutos.csv'
OUTPUT_DIR = '../Data Layer/silver'
OUTPUT_CSV = 'dados_silver.csv'

# Configuração do banco de dados PostgreSQL (Docker)
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5433'),
    'database': os.getenv('DB_NAME', 'grupo08'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres')
}

print('Configuração definida:')
print(f'   Input: {INPUT_FILE}')
print(f'   Output: {OUTPUT_DIR}/{OUTPUT_CSV}')
print(f"   Banco: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")

Configuração definida:
   Input: ../Data Layer/raw/dados_brutos.csv
   Output: ../Data Layer/silver/dados_silver.csv
   Banco: localhost:5433/grupo08


## 2. Definição de Colunas

### Colunas Removidas e Justificativas

Com base na persona do **Diretor de Estratégia de Investimentos**, as seguintes colunas foram **removidas** por não agregarem valor às análises financeiras:

| Coluna | Motivo da Remoção |
|--------|------------------|
| `backdrop_path` | URL de imagem - irrelevante para análise financeira |
| `poster_path` | URL de imagem - irrelevante para análise financeira |
| `homepage` | Link externo - não contribui para decisões de investimento |
| `imdb_id` | Identificador externo - já temos `id` como chave primária |
| `overview` | Texto longo (sinópse) - não quantificável para análises |
| `tagline` | Texto de marketing - subjetivo e não mensurável |
| `keywords` | Lista de palavras-chave - texto não estruturado |
| `spoken_languages` | Idiomas falados - baixo impacto em análises financeiras |
| `adult` | Filtragem já aplicada (removemos conteúdo adulto) |

In [3]:
# =============================================================================
# DEFINIÇÃO DE COLUNAS
# =============================================================================

# Colunas a serem REMOVIDAS (irrelevantes para análise de investimento)
COLUNAS_REMOVIDAS = [
    'backdrop_path',    # URL de imagem
    'poster_path',      # URL de imagem
    'homepage',         # Link externo
    'imdb_id',          # ID duplicado
    'overview',         # Texto longo
    'tagline',          # Texto de marketing
    'keywords',         # Texto não estruturado
    'spoken_languages', # Baixo impacto financeiro
    'adult'             # Filtrado na transformação
]

# Colunas MANTIDAS da base original
COLUNAS_MANTIDAS = [
    'id', 'title', 'original_title', 'original_language',
    'release_date', 'status', 'runtime', 'budget', 'revenue',
    'vote_average', 'vote_count', 'popularity',
    'genres', 'production_companies', 'production_countries'
]

print(f'Colunas removidas: {len(COLUNAS_REMOVIDAS)}')
print(f'Colunas mantidas: {len(COLUNAS_MANTIDAS)}')

Colunas removidas: 9
Colunas mantidas: 15


## 3. Extract - Carregamento dos Dados Brutos

In [4]:
# =============================================================================
# EXTRACT - CARREGAMENTO DOS DADOS
# =============================================================================

print('=' * 70)
print('ETAPA 1: EXTRACT')
print('=' * 70)

print(f'\nCarregando dados de: {INPUT_FILE}')

# Verificação do arquivo de entrada
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f'Arquivo não encontrado: {INPUT_FILE}')

try:
    df_raw = pd.read_csv(INPUT_FILE, encoding='utf-8', low_memory=False)
except Exception as e:
    raise Exception(f'Erro ao ler arquivo CSV: {e}')

print(f'\nDados carregados com sucesso!')
print(f'   Registros: {len(df_raw):,}')
print(f'   Colunas: {len(df_raw.columns)}')
print(f'   Memória: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB')
print(f'\nColunas disponíveis:')
print(list(df_raw.columns))

ETAPA 1: EXTRACT

Carregando dados de: ../Data Layer/raw/dados_brutos.csv

Dados carregados com sucesso!
   Registros: 1,353,976
   Colunas: 24
   Memória: 1457.81 MB

Colunas disponíveis:
['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'tagline', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'keywords']


In [ ]:
# Pré-visualização dos dados brutos
print('\nPrimeiras 5 linhas:')
df_raw.head()

## 4. Transform - Limpeza e Transformação

In [5]:
# =============================================================================
# TRANSFORM - ETAPA 1: SELEÇÃO DE COLUNAS
# =============================================================================

print('=' * 70)
print('ETAPA 2: TRANSFORM')
print('=' * 70)

print('\n[1/6] Selecionando colunas relevantes...')

colunas_existentes = [col for col in COLUNAS_MANTIDAS if col in df_raw.columns]
colunas_faltantes = [col for col in COLUNAS_MANTIDAS if col not in df_raw.columns]

if colunas_faltantes:
    print(f'   AVISO: Colunas não encontradas: {colunas_faltantes}')

df = df_raw[colunas_existentes].copy()

print(f'   Colunas selecionadas: {len(colunas_existentes)}')
print(f'   Colunas removidas: {len(df_raw.columns) - len(colunas_existentes)}')
print(f'   Shape atual: {df.shape}')

ETAPA 2: TRANSFORM

[1/6] Selecionando colunas relevantes...
   Colunas selecionadas: 15
   Colunas removidas: 9
   Shape atual: (1353976, 15)


In [6]:
# =============================================================================
# TRANSFORM - ETAPA 2: FILTRAGEM DE REGISTROS
# =============================================================================

print('\n[2/6] Filtrando registros...')

registros_inicial = len(df)

# Filtro 1: Apenas filmes lançados
if 'status' in df.columns:
    df = df[df['status'] == 'Released']
    print(f'   Após filtrar Released: {len(df):,} registros')

# Filtro 2: Remover conteúdo adulto
if 'adult' in df_raw.columns:
    ids_adulto = df_raw[df_raw['adult'] == True]['id']
    df = df[~df['id'].isin(ids_adulto)]
    print(f'   Após remover adulto: {len(df):,} registros')

# Filtro 3: Remover duplicatas por ID
duplicatas_antes = df.duplicated(subset=['id'], keep='first').sum()
df = df.drop_duplicates(subset=['id'], keep='first')
print(f'   Duplicatas removidas: {duplicatas_antes:,}')

registros_removidos = registros_inicial - len(df)
print(f'\n   Total removido: {registros_removidos:,} ({registros_removidos/registros_inicial*100:.1f}%)')
print(f'   Registros restantes: {len(df):,}')


[2/6] Filtrando registros...
   Após filtrar Released: 1,309,195 registros
   Após remover adulto: 1,175,485 registros
   Duplicatas removidas: 898

   Total removido: 179,389 (13.2%)
   Registros restantes: 1,174,587


In [7]:
# =============================================================================
# TRANSFORM - ETAPA 3: CONVERSÃO DE TIPOS
# =============================================================================

print('\n[3/6] Convertendo tipos de dados...')

colunas_numericas = ['budget', 'revenue', 'runtime', 'vote_average', 'vote_count', 'popularity']

for col in colunas_numericas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(f'   Colunas numéricas convertidas: {len(colunas_numericas)}')

if 'release_date' in df.columns:
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
    print("   Coluna 'release_date' convertida para datetime")

colunas_string = ['title', 'original_title', 'original_language', 'status',
                  'genres', 'production_companies', 'production_countries']

for col in colunas_string:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].replace(['nan', 'None', ''], pd.NA)

print(f'   Colunas string limpas: {len(colunas_string)}')


[3/6] Convertendo tipos de dados...
   Colunas numéricas convertidas: 6
   Coluna 'release_date' convertida para datetime
   Colunas string limpas: 7


In [8]:
# =============================================================================
# TRANSFORM - ETAPA 4: TRATAMENTO DE VALORES ZERADOS/NULOS
# =============================================================================

print('\n[4/8] Tratando valores zerados e nulos...')

if 'budget' in df.columns:
    zeros_budget = (df['budget'] == 0).sum()
    df.loc[df['budget'] == 0, 'budget'] = pd.NA
    print(f'   Budget: {zeros_budget:,} zeros convertidos para nulo')

if 'revenue' in df.columns:
    zeros_revenue = (df['revenue'] == 0).sum()
    df.loc[df['revenue'] == 0, 'revenue'] = pd.NA
    print(f'   Revenue: {zeros_revenue:,} zeros convertidos para nulo')

if 'runtime' in df.columns:
    zeros_runtime = (df['runtime'] == 0).sum()
    df.loc[df['runtime'] == 0, 'runtime'] = pd.NA
    print(f'   Runtime: {zeros_runtime:,} zeros convertidos para nulo')

registros_antes = len(df)
df = df[df['vote_count'].notna() | df['budget'].notna() | df['revenue'].notna()]
print(f'   Registros sem nenhum dado relevante removidos: {registros_antes - len(df):,}')


[4/8] Tratando valores zerados e nulos...
   Budget: 1,104,767 zeros convertidos para nulo
   Revenue: 1,151,839 zeros convertidos para nulo
   Runtime: 346,263 zeros convertidos para nulo
   Registros sem nenhum dado relevante removidos: 0


In [9]:
# =============================================================================
# TRANSFORM - ETAPA 5: EXTRAÇÃO DE CAMPOS COMPOSTOS
# =============================================================================

print('\n[5/8] Extraindo campos compostos...')

if 'genres' in df.columns:
    df['primary_genre'] = df['genres'].apply(
        lambda x: str(x).split(',')[0].strip() if pd.notna(x) and str(x) != 'nan' else None
    )
    print(f"   Gêneros primários extraídos: {df['primary_genre'].nunique()} únicos")

if 'production_companies' in df.columns:
    df['primary_company'] = df['production_companies'].apply(
        lambda x: str(x).split(',')[0].strip() if pd.notna(x) and str(x) != 'nan' else None
    )
    print(f"   Produtoras primárias extraídas: {df['primary_company'].nunique()} únicas")

if 'production_countries' in df.columns:
    df['primary_country'] = df['production_countries'].apply(
        lambda x: str(x).split(',')[0].strip() if pd.notna(x) and str(x) != 'nan' else None
    )
    print(f"   Países primários extraídos: {df['primary_country'].nunique()} únicos")


[5/8] Extraindo campos compostos...
   Gêneros primários extraídos: 19 únicos
   Produtoras primárias extraídas: 130706 únicas
   Países primários extraídos: 245 únicos


In [10]:
# =============================================================================
# TRANSFORM - ETAPA 6: CRIAÇÃO DE CAMPOS TEMPORAIS
# =============================================================================

print('\n[6/8] Criando campos temporais...')

if 'release_date' in df.columns:
    df['release_year'] = df['release_date'].dt.year.astype('Int64')
    df['release_month'] = df['release_date'].dt.month.astype('Int64')
    df['release_decade'] = (df['release_year'] // 10 * 10).astype('Int64')
    
    meses_pt = {1: 'Janeiro', 2: 'Fevereiro', 3: 'Marco', 4: 'Abril',
                5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
                9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'}
    df['release_month_name'] = df['release_month'].map(meses_pt)
    
    df['release_day_of_week'] = df['release_date'].dt.dayofweek
    dias_pt = {0: 'Segunda', 1: 'Terca', 2: 'Quarta', 3: 'Quinta',
               4: 'Sexta', 5: 'Sabado', 6: 'Domingo'}
    df['release_day_name'] = df['release_day_of_week'].map(dias_pt)
    
    print(f'   Campos temporais criados')
    print(f"   Período dos dados: {df['release_year'].min()} a {df['release_year'].max()}")


[6/8] Criando campos temporais...
   Campos temporais criados
   Período dos dados: 1800 a 2061


In [11]:
# =============================================================================
# TRANSFORM - ETAPA 7: CRIAÇÃO DE MÉTRICAS FINANCEIRAS
# =============================================================================

print('\n[7/8] Criando métricas financeiras...')

# Lucro absoluto
df['profit'] = df['revenue'] - df['budget']
print("   Campo 'profit' criado")

# ROI (Return on Investment) em percentual
df['roi'] = np.where(
    (df['budget'].notna()) & (df['budget'] > 0),
    ((df['revenue'] - df['budget']) / df['budget']) * 100,
    pd.NA
)
df['roi'] = pd.to_numeric(df['roi'], errors='coerce')
print("   Campo 'roi' criado")

# Flag de lucratividade
df['is_profitable'] = np.where(
    (df['revenue'].notna()) & (df['budget'].notna()),
    df['revenue'] > df['budget'],
    pd.NA
)
print("   Campo 'is_profitable' criado")

# Faixas de orçamento
def categorize_budget(budget):
    if pd.isna(budget):
        return None
    elif budget < 1_000_000:
        return 'Micro (< $1M)'
    elif budget < 10_000_000:
        return 'Baixo ($1M - $10M)'
    elif budget < 50_000_000:
        return 'Medio ($10M - $50M)'
    elif budget < 100_000_000:
        return 'Alto ($50M - $100M)'
    elif budget < 200_000_000:
        return 'Muito Alto ($100M - $200M)'
    else:
        return 'Blockbuster ($200M+)'

df['budget_tier'] = df['budget'].apply(categorize_budget)
print("   Campo 'budget_tier' criado")

filmes_com_roi = df['roi'].notna().sum()
filmes_lucrativos = (df['is_profitable'] == True).sum()

print(f'\n   Filmes com ROI calculável: {filmes_com_roi:,}')
if filmes_com_roi > 0:
    taxa_sucesso = filmes_lucrativos / filmes_com_roi * 100
    print(f'   Taxa de sucesso (lucrativos): {taxa_sucesso:.1f}%')
    print(f"   ROI médio: {df['roi'].mean():.1f}%")
    print(f"   ROI mediano: {df['roi'].median():.1f}%")


[7/8] Criando métricas financeiras...
   Campo 'profit' criado
   Campo 'roi' criado
   Campo 'is_profitable' criado
   Campo 'budget_tier' criado

   Filmes com ROI calculável: 15,979
   Taxa de sucesso (lucrativos): 59.1%
   ROI médio: 64126538.2%
   ROI mediano: 49.0%


In [12]:
# =============================================================================
# TRANSFORM - ETAPA 8: ORDENAÇÃO E SELEÇÃO FINAL
# =============================================================================

print('\n[8/8] Finalizando transformações...')

colunas_ordenadas = [
    'id', 'title', 'original_title', 'original_language',
    'release_date', 'release_year', 'release_month', 'release_month_name',
    'release_day_of_week', 'release_day_name', 'release_decade',
    'runtime', 'status', 'genres', 'primary_genre',
    'production_companies', 'primary_company',
    'production_countries', 'primary_country',
    'vote_average', 'vote_count', 'popularity',
    'budget', 'revenue', 'profit', 'roi', 'is_profitable', 'budget_tier'
]

colunas_finais = [col for col in colunas_ordenadas if col in df.columns]
df_silver = df[colunas_finais].copy()

df_silver = df_silver.sort_values('popularity', ascending=False).reset_index(drop=True)

print(f'\n   Shape final: {df_silver.shape}')
print(f'   Colunas finais: {len(colunas_finais)}')


[8/8] Finalizando transformações...

   Shape final: (1174587, 28)
   Colunas finais: 28


In [13]:
# =============================================================================
# RESUMO DA TRANSFORMAÇÃO
# =============================================================================

print('\n' + '=' * 70)
print('RESUMO DA TRANSFORMAÇÃO')
print('=' * 70)

print(f'\nDADOS:')
print(f'   Registros originais: {len(df_raw):,}')
print(f'   Registros Silver: {len(df_silver):,}')
print(f'   Redução: {len(df_raw) - len(df_silver):,}')

print(f'\nCOLUNAS:')
print(f'   Originais: {len(df_raw.columns)}')
print(f'   Silver: {len(df_silver.columns)}')

print(f'\nFILMES:')
print(f"   Gêneros únicos: {df_silver['primary_genre'].nunique()}")
print(f"   Produtoras únicas: {df_silver['primary_company'].nunique()}")
print(f"   Países únicos: {df_silver['primary_country'].nunique()}")


RESUMO DA TRANSFORMAÇÃO

DADOS:
   Registros originais: 1,353,976
   Registros Silver: 1,174,587
   Redução: 179,389

COLUNAS:
   Originais: 24
   Silver: 28

FILMES:
   Gêneros únicos: 19
   Produtoras únicas: 130706
   Países únicos: 245


In [ ]:
# Visualização das primeiras linhas
print('\nAmostra dos dados Silver:')
df_silver.head(10)

## 5. Load - Salvamento dos Dados

In [14]:
# =============================================================================
# LOAD - ETAPA 1: SALVAR CSV (BACKUP)
# =============================================================================

print('=' * 70)
print('ETAPA 3: LOAD')
print('=' * 70)

print('\n[1/2] Salvando backup CSV...')

os.makedirs(OUTPUT_DIR, exist_ok=True)

csv_path = os.path.join(OUTPUT_DIR, OUTPUT_CSV)
df_silver.to_csv(csv_path, index=False, encoding='utf-8')

tamanho_mb = os.path.getsize(csv_path) / 1024**2

print(f'   Arquivo: {csv_path}')
print(f'   Tamanho: {tamanho_mb:.2f} MB')
print(f'   Registros: {len(df_silver):,}')

ETAPA 3: LOAD

[1/2] Salvando backup CSV...
   Arquivo: ../Data Layer/silver\dados_silver.csv
   Tamanho: 190.19 MB
   Registros: 1,174,587


In [19]:
# =============================================================================
# LOAD - ETAPA 2: CRIAR SCHEMA E TABELA NO POSTGRESQL
# =============================================================================

print('\n[2/2] Carregando no PostgreSQL...')

DDL_SCHEMA = 'CREATE SCHEMA IF NOT EXISTS silver;'

DDL_TABLE = '''
DROP TABLE IF EXISTS silver.filmes CASCADE;

CREATE TABLE silver.filmes (
    id                    INTEGER PRIMARY KEY,
    title                 TEXT,
    original_title        TEXT,
    original_language     VARCHAR(10),
    release_date          DATE,
    release_year          INTEGER,
    release_month         INTEGER,
    release_month_name    VARCHAR(20),
    release_day_of_week   INTEGER,
    release_day_name      VARCHAR(20),
    release_decade        INTEGER,
    runtime               INTEGER,
    status                VARCHAR(50),
    genres                TEXT,
    primary_genre         VARCHAR(100),
    production_companies  TEXT,
    primary_company       TEXT,
    production_countries  TEXT,
    primary_country       VARCHAR(100),
    vote_average          NUMERIC(4,2),
    vote_count            INTEGER,
    popularity            DOUBLE PRECISION,
    budget                BIGINT,
    revenue               BIGINT,
    profit                BIGINT,
    roi                   DOUBLE PRECISION,
    is_profitable         BOOLEAN,
    budget_tier           VARCHAR(50),
    created_at            TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE INDEX idx_filmes_year ON silver.filmes(release_year);
CREATE INDEX idx_filmes_genre ON silver.filmes(primary_genre);
CREATE INDEX idx_filmes_budget_tier ON silver.filmes(budget_tier);
CREATE INDEX idx_filmes_popularity ON silver.filmes(popularity DESC);
'''

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    cur.execute(DDL_SCHEMA)
    print("   Schema 'silver' criado/verificado")
    
    cur.execute(DDL_TABLE)
    print("   Tabela 'silver.filmes' criada")
    
    conn.commit()
    cur.close()
    conn.close()
    
except Exception as e:
    print(f'\n   AVISO: Não foi possível conectar ao banco: {e}')
    print('   Verifique se o Docker está rodando: docker-compose up -d')


[2/2] Carregando no PostgreSQL...
   Schema 'silver' criado/verificado
   Tabela 'silver.filmes' criada


In [20]:
# =============================================================================
# LOAD - INSERÇÃO DOS DADOS
# =============================================================================

def prepare_value(val):
    if pd.isna(val):
        return None
    if isinstance(val, (np.integer, np.floating)):
        if np.isnan(val) or np.isinf(val):
            return None
        return float(val) if isinstance(val, np.floating) else int(val)
    if isinstance(val, pd.Timestamp):
        return val.date()
    return val

try:
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    print('\n   Preparando dados para inserção...')
    
    colunas_insert = [
        'id', 'title', 'original_title', 'original_language',
        'release_date', 'release_year', 'release_month', 'release_month_name',
        'release_day_of_week', 'release_day_name', 'release_decade',
        'runtime', 'status', 'genres', 'primary_genre',
        'production_companies', 'primary_company',
        'production_countries', 'primary_country',
        'vote_average', 'vote_count', 'popularity',
        'budget', 'revenue', 'profit', 'roi', 'is_profitable', 'budget_tier'
    ]
    
    data = []
    for _, row in df_silver.iterrows():
        record = tuple(prepare_value(row.get(col)) for col in colunas_insert)
        data.append(record)
    
    placeholders = ', '.join(['%s'] * len(colunas_insert))
    insert_sql = f'''
        INSERT INTO silver.filmes ({', '.join(colunas_insert)})
        VALUES ({placeholders})
        ON CONFLICT (id) DO NOTHING
    '''
    
    print(f'   Inserindo {len(data):,} registros...')
    execute_batch(cur, insert_sql, data, page_size=1000)
    
    conn.commit()
    
    cur.execute('SELECT COUNT(*) FROM silver.filmes')
    count = cur.fetchone()[0]
    print(f'   Registros inseridos: {count:,}')
    
    cur.close()
    conn.close()
    
    print('\n   Dados carregados com sucesso no PostgreSQL!')
    
except Exception as e:
    print(f'\n   ERRO ao inserir dados: {e}')
    print('   Os dados foram salvos no CSV de backup.')


   Preparando dados para inserção...
   Inserindo 1,174,587 registros...
   Registros inseridos: 1,174,587

   Dados carregados com sucesso no PostgreSQL!


In [21]:
# =============================================================================
# VERIFICAÇÃO FINAL
# =============================================================================

print('\n' + '=' * 70)
print('VERIFICAÇÃO FINAL')
print('=' * 70)

try:
    conn = psycopg2.connect(**DB_CONFIG)
    
    print('\nAmostra dos dados no banco:')
    query_sample = '''
        SELECT id, title, release_year, primary_genre, budget, revenue, roi
        FROM silver.filmes
        ORDER BY popularity DESC
        LIMIT 10
    '''
    df_sample = pd.read_sql(query_sample, conn)
    display(df_sample)
    
    print('\nEstatísticas agregadas:')
    query_stats = '''
        SELECT 
            COUNT(*) as total_filmes,
            COUNT(DISTINCT primary_genre) as generos,
            COUNT(DISTINCT primary_company) as produtoras,
            MIN(release_year) as ano_min,
            MAX(release_year) as ano_max,
            AVG(roi) as roi_medio,
            SUM(CASE WHEN is_profitable THEN 1 ELSE 0 END) as filmes_lucrativos
        FROM silver.filmes
    '''
    df_stats = pd.read_sql(query_stats, conn)
    display(df_stats)
    
    conn.close()
    
except Exception as e:
    print(f'Não foi possível verificar o banco: {e}')


VERIFICAÇÃO FINAL

Amostra dos dados no banco:


,id,title,release_year,primary_genre,budget,revenue,roi
0,565770,Blue Beetle,2023,Action,120000000,124818235,4.015196
1,980489,Gran Turismo,2023,Action,60000000,114800000,91.333333
2,968051,The Nun II,2023,Horror,38500000,231200000,500.519481
3,615656,Meg 2: The Trench,2023,Action,129000000,384056482,197.718203
4,762430,Retribution,2023,Action,20000000,12905464,-35.472680
5,1008042,Talk to Me,2023,Horror,4500000,72600000,1513.333333
6,385687,Fast X,2023,Action,340000000,704709660,107.267547
7,678512,Sound of Freedom,2023,Action,15000000,212587173,1317.247820
8,346698,Barbie,2023,Comedy,145000000,1428545028,885.203468
9,976573,Elemental,2023,Animation,200000000,486797988,143.398994



Estatísticas agregadas:


,total_filmes,generos,produtoras,ano_min,ano_max,roi_medio,filmes_lucrativos
0,1174587,19,130706,1800,2061,6.412654e+07,9447


## 6. Conclusão

O processo ETL Raw → Silver foi concluído com sucesso!

**Transformações realizadas:**
- Remoção de colunas irrelevantes (URLs, textos longos, IDs duplicados)
- Filtragem de filmes não lançados e conteúdo adulto
- Tratamento de valores zerados/nulos
- Criação de métricas financeiras (profit, ROI, budget_tier)
- Extração de campos compostos (gênero, produtora, país)
- Criação de campos temporais (ano, mês, década)

**Próximos passos:**
1. Execute o notebook `etl_silver_to_gold.ipynb` para criar o Data Warehouse dimensional
2. Explore os dados na camada Silver usando `Data Layer/silver/analytics.ipynb`